<h1>Regex with Hacker News titles</h1>
<h4>Capturing various languages mentioned in titles from Hacker News</h4>

In [1]:
#Dataset from https://www.kaggle.com/hacker-news/hacker-news-posts

import pandas as pd
import re

hn = pd.read_csv("hacker_news.csv")
titles = hn["title"]

hn.head()

,id,title,url,num_points,num_comments,author,created_at
0,12224879,Interactive Dynamic Video,http://www.interactivedynamicvideo.com/,386,52,ne0phyte,8/4/2016 11:52
1,11964716,Florida DJs May Face Felony for April Fools' W...,http://www.thewire.com/entertainment/2013/04/f...,2,1,vezycash,6/23/2016 22:20
2,11919867,Technology ventures: From Idea to Enterprise,https://www.amazon.com/Technology-Ventures-Ent...,3,1,hswarna,6/17/2016 0:01
3,10301696,Note by Note: The Making of Steinway L1037 (2007),http://www.nytimes.com/2007/11/07/movies/07ste...,8,2,walterbell,9/30/2015 4:12
4,10482257,Title II kills investment? Comcast and other I...,http://arstechnica.com/business/2015/10/comcas...,53,22,Deinos,10/31/2015 9:48


<h4>Python</h4>

In [2]:
#Collect all the titles that contain Python or python
pattern = r"python"
py_titles = titles[titles.str.contains(pattern, flags=re.I)]

print(py_titles.head())

102                  From Python to Lua: Why We Switched
103            Ubuntu 16.04 LTS to Ship Without Python 2
144    Create a GUI Application Using Qt and Python i...
196    How I Solved GCHQ's Xmas Card with Python and ...
436    Unikernel Power Comes to Java, Node.js, Go, an...
Name: title, dtype: object


In [3]:
#Count all the titles that contain Python or python
py_titles_counts = titles.str.contains(pattern, flags=re.I).sum()

print(py_titles_counts)

160


In [4]:
#Capture groups to extract the version of Python that is mentioned
pattern = r"[Pp]ython ([\d\.]+)"

py_versions = titles.str.extract(pattern)

#Create a dictionary frequancy table of the extracted Python versions
py_versions_freq = dict(py_versions[0].value_counts())

print(py_versions_freq)

{'3': 10, '3.5': 3, '2': 3, '3.6': 2, '8': 1, '1.5': 1, '3.5.0': 1, '2.7': 1, '4': 1}


<h4>Java</h4>

In [5]:
#Collect all the titles that contain Java or java (but not JavaScript)
pattern = r"\bjava\b"
java_titles = titles[titles.str.contains(pattern, flags=re.I)]

print(java_titles.head())

436     Unikernel Power Comes to Java, Node.js, Go, an...
811     Ask HN: Are there any projects or compilers wh...
1023                         Pippo  Web framework in Java
1972          Node.js vs. Java: Which Is Faster for APIs?
2093                    Java EE and Microservices in 2016
Name: title, dtype: object


<h4>SQL</h4>

In [6]:
#Count all variations on SQL (also Sql and sql)
pattern = r"sql"
sql_counts = titles.str.contains(pattern, flags=re.I).sum()
print(sql_counts)

108


In [7]:
#Collect all variations on SQL (also Sql and sql) and put them in a frequency table
pattern = r"(sql)"
sql_capitalizations = titles.str.extract(pattern, flags=re.I)
sql_capitalizations_freq = sql_capitalizations[0].value_counts()
print(sql_capitalizations_freq)

SQL    101
Sql      4
sql      3
Name: 0, dtype: int64


In [18]:
#Replace all variations on SQL (also Sql and sql) with SQL
pattern = r"(sql)"

titles_clean = titles.str.replace(pattern, "SQL", flags=re.I)

#Check
sql_titles = titles_clean.str.extract(pattern, flags=re.I)
sql_freq = sql_titles[0].value_counts()
print(sql_upper_freq)

SQL    108
Name: 0, dtype: int64


In [19]:
#Extact all variations on SQL also PostgreSQL, mySQL etc
pattern = r"(\w+SQL)"
sql_flavors = titles_clean.str.extract(pattern, flags=re.I)
sql_flavors_freq = sql_flavors[0].value_counts()
print(sql_flavors_freq)

PostgreSQL    27
NoSQL         16
MySQL         12
mySQL          1
noSQL          1
SparkSQL       1
MemSQL         1
CloudSQL       1
Name: 0, dtype: int64


In [27]:
#Extact all mentions of SQL into a new column, make them all lowercase (to avoid duplicates)
hn_sql = hn[titles_clean.str.contains(r"\w+SQL", flags=re.I)].copy()

hn_sql["flavor"] = hn_sql["title"].str.extract(r"(\w+SQL)", flags=re.I)
hn_sql["flavor"] = hn_sql["flavor"].str.lower()

#Average number of comments for each flavor
sql_pivot = hn_sql.pivot_table(index="flavor", values="num_comments")
print(sql_pivot)

            num_comments
flavor                  
cloudsql        5.000000
memsql         14.000000
mysql          12.230769
nosql          14.529412
postgresql     25.962963
sparksql        1.000000


<h4>C</h4>

In [11]:
#Pattern for C that won't include C++ or C.E.O
pattern = r"\b[Cc]\b(?![\+\.])"

#Collect all the titles that contain C
c_titles = titles[titles.str.contains(pattern)]

print(c_titles.head(10))

221                  MemSQL (YC W11) Raises $36M Series C
365                      The new C standards are worth it
444           Moz raises $10m Series C from Foundry Group
521          Fuchsia: Micro kernel written in C by Google
1307            Show HN: Yupp, yet another C preprocessor
1326                     The C standard formalized in Coq
1365                          GNU C Library 2.23 released
1429    Cysignals: signal handling (SIGINT, SIGSEGV, )...
1620                        SDCC  Small Device C Compiler
1949    Rewriting a Ruby C Extension in Rust: How a Na...
Name: title, dtype: object


In [12]:
c_titles_count = titles.str.contains(pattern).sum()
print(c_titles_count)

105


In [13]:
#Also exclude patterns that have the word 'Series'
pattern = r"(?<!Series\s)\b[Cc]\b(?![\+\.])"

#Collect all the titles that contain C
c_titles = titles[titles.str.contains(pattern)]

print(c_titles.head(10))

365                      The new C standards are worth it
521          Fuchsia: Micro kernel written in C by Google
1307            Show HN: Yupp, yet another C preprocessor
1326                     The C standard formalized in Coq
1365                          GNU C Library 2.23 released
1429    Cysignals: signal handling (SIGINT, SIGSEGV, )...
1620                        SDCC  Small Device C Compiler
1949    Rewriting a Ruby C Extension in Rust: How a Na...
2195    MyHTML  HTML Parser on Pure C with POSIX Threa...
2589    Phalcon  PHP framework delivered as a C extension
Name: title, dtype: object


In [14]:
c_titles_count = titles.str.contains(pattern).sum()
print(c_titles_count)

102


<h4>Capturing Domains</h4>

In [38]:
#Collect all domains
pattern = r"\w+://([\w\-\.]+)"

domains = hn["url"].str.extract(pattern, flags=re.I)
domains = domains[0].value_counts()

print(domains.head())

github.com             1008
medium.com              825
www.nytimes.com         525
www.theguardian.com     248
techcrunch.com          245
Name: 0, dtype: int64


In [39]:
#Split different URL parts
#0: protocol, 1: domain, 2: page path
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"

url_parts = hn["url"].str.extract(pattern, flags=re.I)

print(url_parts.head())

  protocol                           domain  \
0     http  www.interactivedynamicvideo.com   
1     http                  www.thewire.com   
2    https                   www.amazon.com   
3     http                  www.nytimes.com   
4     http                  arstechnica.com   

                                                path  
0                                                     
1  entertainment/2013/04/florida-djs-april-fools-...  
2  Technology-Ventures-Enterprise-Thomas-Byers/dp...  
3                2007/11/07/movies/07stein.html?_r=0  
4  business/2015/10/comcast-and-other-isps-boost-...  
